# LAS Conversion Pipeline 

## SemanticKITTI Style (.bin & .label)

In [ ]:
# Paths
velodyne_path = "/path/to/velodyne/"
labels_path = "/path/to/labels/"

# Files
bin_files = sorted(glob.glob(os.path.join(velodyne_path, "*.bin")))
label_files = sorted(glob.glob(os.path.join(labels_path, "*.label")))

In [ ]:
# Load & apply poses
poses = np.loadtxt("/path/to/pose.txt").reshape(-1, 3, 4)  # shape (N, 3, 4)
poses_hom = np.zeros((poses.shape[0], 4, 4))
poses_hom[:, :3, :4] = poses
poses_hom[:, 3, 3] = 1.0 


def transform_points(points, pose):
    ones = np.ones((points.shape[0], 1))
    points_hom = np.hstack([points, ones])  # (N, 4)
    transformed = (pose @ points_hom.T).T
    return transformed[:, :3]


In [ ]:
# Lists for every field of the combined sequence of .bin and .label for a standard point_format=3 LAS file
all_x = []
all_y = []
all_z = []
all_intensity = []
all_semantic = []
all_instance = []

In [ ]:
for idx, (bin_file, label_file) in enumerate(zip(bin_files, label_files)):
    points = np.fromfile(bin_file, dtype=np.float32).reshape(-1, 4)
    labels = np.fromfile(label_file, dtype=np.uint32)

    points = np.nan_to_num(points, nan=0.0) # comment this if not needed
    semantic = labels & 0xFFFF
    instance = labels >> 16

    # LAS classification field requires 5 bits, so a remap might be needed
    mapping = {0: 0, 1: 1, 10: 2, 30: 3, 40: 4, 70: 5, 72: 6, 80: 7}  # Aircloud remap
    mapped_labels = np.vectorize(mapping.get)(semantic)
    transformed = transform_points(points[:,:3], poses_hom[idx])
    
    all_x.append(transformed[:, 0])
    all_y.append(transformed[:, 1])
    all_z.append(transformed[:, 2])
    all_intensity.append((points[:, 3]).astype(np.uint16))
    all_semantic.append(mapped_labels)
    all_instance.append(instance % 256)  # for user_data



In [ ]:
# Stack all together
x = np.concatenate(all_x)
y = np.concatenate(all_y)
z = np.concatenate(all_z)
intensity = np.concatenate(all_intensity)
classification = np.concatenate(all_semantic)
user_data = np.concatenate(all_instance).astype(np.uint8)

In [ ]:
# Writing the LAS fields
header = laspy.LasHeader(point_format=3, version="1.4")
header.x_scale = header.y_scale = header.z_scale = 0.01
las = laspy.LasData(header)
las.x = x
las.y = y
las.z = z
las.intensity = intensity
las.classification = classification
las.user_data = user_data

las.write("my_file.las")

In case of lack of poses.txt, look for x/y/z_offset fields for manual configuration

In [ ]:
# Testing
las_file = laspy.read("my_file.las")
las_file.header, las_file.point_format.dimension_names